In [3]:
import os
import pandas as pd

In [2]:
import os
import glob
import gzip
import xml.etree.ElementTree as ET
import pandas as pd

In [3]:
root_folder = 'iTIC-Longdo-Traffic-events-2023-incomplete'

def flatten_item(item):
    row = {}
    for child in item:
        row[child.tag] = child.text
    return row

# Find all .gz files recursively

all_gz_files = []
for root, dirs, files in os.walk(root_folder):
    for f in files:
        if f.lower().endswith(".xml.gz"):
            all_gz_files.append(os.path.join(root, f))

# Filter out TMC/traffic files (by filename)
gz_files = [
    f for f in all_gz_files
    if "tmc" not in os.path.basename(f).lower()
]

all_rows = []


for gz_file in gz_files:
    try:
        # Decompress the file and read XML content
        with gzip.open(gz_file, 'rb') as f:
            xml_data = f.read()

        # Parse XML from the decompressed content
        root = ET.fromstring(xml_data)

        # Extract all <item> elements
        items = root.findall(".//channel/item")

        for item in items:
            row = flatten_item(item)
            row["source_file"] = gz_file
            all_rows.append(row)

    except Exception as e:
        print(f"❌ Error parsing {gz_file}: {e}")

# Save to CSV
df = pd.DataFrame(all_rows)

# filter needed columns
# df = df[df["type"].astype(int) == 3]
# df = df[["latitude", "longitude", "starttime", "description_en", "road_en"]]

# df.to_csv("2021_incidents.csv", index=False)


❌ Error parsing iTIC-Longdo-Traffic-events-2023-incomplete/03/29/202303291730-event.xml.gz: no element found: line 1, column 0
❌ Error parsing iTIC-Longdo-Traffic-events-2023-incomplete/03/29/202303291700-event.xml.gz: no element found: line 1, column 0
❌ Error parsing iTIC-Longdo-Traffic-events-2023-incomplete/03/29/202303291800-event.xml.gz: no element found: line 1, column 0
❌ Error parsing iTIC-Longdo-Traffic-events-2023-incomplete/03/28/202303281730-event.xml.gz: no element found: line 1, column 0
❌ Error parsing iTIC-Longdo-Traffic-events-2023-incomplete/03/15/202303150100-event.xml.gz: no element found: line 1, column 0
❌ Error parsing iTIC-Longdo-Traffic-events-2023-incomplete/05/30/202305301730-event.xml.gz: no element found: line 1, column 0
❌ Error parsing iTIC-Longdo-Traffic-events-2023-incomplete/02/23/202302232330-event.xml.gz: no element found: line 1, column 0
❌ Error parsing iTIC-Longdo-Traffic-events-2023-incomplete/07/01/202307011400-event.xml.gz: no element found: l

In [7]:
df.columns

Index(['title', 'title_en', 'link', 'eid', 'latitude', 'longitude', 'type',
       'description', 'description_en', 'starttime', 'starttimestamp',
       'stoptime', 'stoptimestamp', 'allday', 'status', 'pubDate', 'image',
       'imagecount', 'commentcount', 'contributor', 'road', 'road_en',
       'source_file'],
      dtype='object')

In [6]:
df = df[df["type"].astype(int) == 3]
df.to_csv('sample.csv')